In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math
import serial
import time

# --- 1. THE SMOOTHING FILTER (One Euro Filter) ---
# This class removes jitter from your hand coordinates
class OneEuroFilter:
    def __init__(self, t0, x0, min_cutoff=1.0, beta=0.0):
        self.t_prev = t0
        self.x_prev = np.array(x0, dtype=float)
        self.dx_prev = np.zeros_like(x0, dtype=float)
        self.min_cutoff = min_cutoff
        self.beta = beta

    def smoothing_factor(self, t_e, cutoff):
        r = 2 * math.pi * cutoff * t_e
        return r / (r + 1)

    def exponential_smoothing(self, a, x, x_prev):
        return a * x + (1 - a) * x_prev

    def __call__(self, t, x):
        t_e = t - self.t_prev
        if t_e <= 0: return self.x_prev # Prevent divide by zero
        
        a_d = self.smoothing_factor(t_e, self.min_cutoff)
        dx = (x - self.x_prev) / t_e
        dx_hat = self.exponential_smoothing(a_d, dx, self.dx_prev)
        
        cutoff = self.min_cutoff + self.beta * np.abs(dx_hat)
        a = self.smoothing_factor(t_e, cutoff)
        x_hat = self.exponential_smoothing(a, x, self.x_prev)
        
        self.x_prev = x_hat
        self.dx_prev = dx_hat
        self.t_prev = t
        return x_hat

print("Cell 1 Complete: Filter Ready.")

Cell 1 Complete: Filter Ready.


In [3]:
# --- 2. INVERSE KINEMATICS SOLVER ---
def solve_ik(x, y, z):
    # ROBOT DIMENSIONS (Update these to match your hardware in cm)
    L_SHOULDER = 9.5  # Length from Shoulder to Elbow
    L_ELBOW = 9.0     # Length from Elbow to Wrist
    
    # 1. BASE ROTATION (Theta 1)
    # Math: simple atan2(y, x) to point towards the object
    theta_base = math.atan2(y, x)
    
    # 2. CALCULATE REACH (Distance from shoulder to target in 2D)
    # 'r' is the horizontal distance, 'z' is the vertical height
    r = math.sqrt(x**2 + y**2)
    
    # 3. SOLVE TRIANGLE (Cosine Rule) for Shoulder & Elbow
    # Hypotenuse (distance from shoulder pivot to wrist)
    c = math.sqrt(r**2 + z**2)
    
    # Safety: Cannot reach further than arm length
    if c > (L_SHOULDER + L_ELBOW):
        c = L_SHOULDER + L_ELBOW - 0.01 # Clamp to max reach
        
    # Alpha: Angle of the hypotenuse
    alpha = math.atan2(z, r)
    
    # Cosine Rule for internal angles
    # acos arguments must be between -1 and 1
    try:
        cos_angle_a = (L_SHOULDER**2 + c**2 - L_ELBOW**2) / (2 * L_SHOULDER * c)
        angle_a = math.acos(cos_angle_a)
        
        cos_angle_b = (L_SHOULDER**2 + L_ELBOW**2 - c**2) / (2 * L_SHOULDER * L_ELBOW)
        angle_b = math.acos(cos_angle_b)
    except ValueError:
        return 90, 90, 90, 90 # Return default safe pose if math fails

    # Calculate final Servo Angles (Radians -> Degrees)
    theta_shoulder = (alpha + angle_a) * (180 / math.pi)
    theta_elbow = (angle_b) * (180 / math.pi) # Often needs adjustment based on servo mounting
    
    # Convert to 0-180 range (Hardware specific adjustments)
    # Adjust these offsets (e.g., +90, 180-) based on how you screwed your horns on
    base_deg = int(math.degrees(theta_base)) + 90 
    shoulder_deg = int(theta_shoulder)
    elbow_deg = int(180 - theta_elbow) # Elbow usually moves opposite
    
    return base_deg, shoulder_deg, elbow_deg

print("Cell 2 Complete: Math Engine Ready.")

Cell 2 Complete: Math Engine Ready.


In [8]:
# --- 3. MAIN EXECUTION LOOP ---

# A. SETUP SERIAL CONNECTION (Connect Arduino first!)
# Replace 'COM3' with your port (Windows) or '/dev/ttyUSB0' (Linux/Mac)
ser = serial.Serial('COM7', 9600, timeout=1) 
time.sleep(2) # Wait for Arduino to reset

# B. SETUP MEDIAPIPE
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# C. INITIALIZE FILTERS
filter_x = OneEuroFilter(time.time(), 0)
filter_y = OneEuroFilter(time.time(), 0)
filter_z = OneEuroFilter(time.time(), 0)

cap = cv2.VideoCapture(0)

print("Starting Loop... Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # 1. Process Image
    frame = cv2.flip(frame, 1) # Mirror effect
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # 2. Extract Wrist Coordinates
            # MediaPipe gives 0.0-1.0. We scale to "Centimeters" roughly.
            # Scaling Factor: 30 means "moving hand across screen = 30cm reach"
            raw_x = (hand_landmarks.landmark[0].x - 0.5) * 40 
            raw_y = (0.8 - hand_landmarks.landmark[0].y) * 30 
            raw_z = (0.5 - hand_landmarks.landmark[0].z) * 20 # Depth estimate
            
            # 3. Smooth the Jitter
            t = time.time()
            smooth_x = filter_x(t, raw_x)
            smooth_y = filter_y(t, raw_y)
            smooth_z = filter_z(t, raw_z)
            
            # 4. Calculate Angles (Inverse Kinematics)
            b, s, e = solve_ik(smooth_x, smooth_y, smooth_z)
            
            # Gripper Logic (Index Finger Tip vs Thumb Tip distance)
            index_tip = hand_landmarks.landmark[8]
            thumb_tip = hand_landmarks.landmark[4]
            dist = math.hypot(index_tip.x - thumb_tip.x, index_tip.y - thumb_tip.y)
            gripper_angle = 180 if dist < 0.05 else 0 # Close if pinched
            
            # 5. Send to Arduino
            # Format: "Base,Shoulder,Elbow,Wrist,Gripper\n"
            # Note: We fix Wrist Rotation to 90 for now
            cmd = f"{b},{s},{e},90,{gripper_angle}\n"
            ser.write(cmd.encode('utf-8'))
            
            # Display text on screen
            cv2.putText(frame, f"Ang: {b},{s},{e}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('AI Robot Teleop', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
ser.close()

Starting Loop... Press 'q' to quit.
